In [1]:
import os
import torch
import time
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from CNN_CNMP import CNN_CNMP
from dataset import GridDataset
import wandb
import math


In [2]:
# Hyperparameters
t_dim = 1                      # step index dimension
grid_size = 32                 # grid size (32x32)

# Try simpler architecture first
cnn_channels = [16, 32, 64]  # Reduced complexity
encoder_hidden_dims = [256, 128]  # Simplified
latent_dim = 128  # Smaller latent space
decoder_hidden_dims = [256, 512]  # Simplified


batch_size = 256
num_epochs = 10000
learning_rate = 0.00005

wandb.init(
    project="ped_forecasting",
    config={
        "learning_rate": learning_rate,
        "architecture": "CNN_CNMP",
        "epochs": num_epochs,
        "batch_siize": batch_size,
        "encoder_hidden_dims": encoder_hidden_dims,
        "latent_dim": latent_dim,
        "decoder_hidden_dims": decoder_hidden_dims,
        "cnn_channels": cnn_channels,
        "grid_size": grid_size
    }
)


wandb: Currently logged in as: arinc-demir (arinc-demir-bo-azi-i-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data generated earlier
data_path = "grids_tensor.pt"
grids_tensor = torch.load(data_path)

# Create the dataset and split into train and validation sets
dataset = GridDataset(grids_tensor, max_encodings=5, max_queries=5)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset = torch.utils.data.Subset(dataset, range(val_size, len(dataset)))
val_dataset = torch.utils.data.Subset(dataset, range(val_size))

# TODO I did this for trying to overfit the model with just one simulation.
#train_dataset = dataset
#val_dataset = dataset

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print(f"Device: {device}")


Device: cuda


/tmp/ipykernel_187065/688883488.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  grids_tensor = torch.load(data_path)


In [4]:
# Initialize the model and optimizer with the CNN component
model = CNN_CNMP(
    t_dim=t_dim,
    grid_size=grid_size,
    encoder_hidden_dims=encoder_hidden_dims,
    decoder_hidden_dims=decoder_hidden_dims,
    latent_dim=latent_dim,
    cnn_channels=cnn_channels
).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)



In [ ]:
start_time = time.time()
best_val_loss = math.inf  # Initialize best validation loss

for epoch in range(num_epochs):
    epoch_start_time = time.time()
    model.train()
    epoch_train_loss = 0.0
    
    for padded_time_indices, padded_grids, encodings_mask, padded_query_indices, padded_query_targets, queries_mask in train_loader:
        # Move data to GPU
        padded_time_indices = padded_time_indices.to(device)
        padded_grids = padded_grids.to(device)
        encodings_mask = encodings_mask.to(device)
        padded_query_indices = padded_query_indices.to(device)
        padded_query_targets = padded_query_targets.to(device)
        queries_mask = queries_mask.to(device)
        
        optimizer.zero_grad()

        # Forward pass with the new model interface
        output = model(padded_time_indices, padded_grids, encodings_mask, padded_query_indices, queries_mask)
        loss = model.loss(output, padded_query_targets, queries_mask)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    avg_train_loss = epoch_train_loss / len(train_loader)
    
    # Add validation every 10 epochs
    if (epoch + 1) % 5 == 0:
        model.eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for padded_time_indices, padded_grids, encodings_mask, padded_query_indices, padded_query_targets, queries_mask in val_loader:
                padded_time_indices = padded_time_indices.to(device)
                padded_grids = padded_grids.to(device)
                encodings_mask = encodings_mask.to(device)
                padded_query_indices = padded_query_indices.to(device)
                padded_query_targets = padded_query_targets.to(device)
                queries_mask = queries_mask.to(device)
                
                output = model(padded_time_indices, padded_grids, encodings_mask, padded_query_indices, queries_mask)
                loss = model.loss(output, padded_query_targets, queries_mask)
                epoch_val_loss += loss.item()
                
        avg_val_loss = epoch_val_loss / len(val_loader)
        wandb.log({"validation_loss": avg_val_loss}, step=epoch)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.8f}")
        
        #scheduler.step(avg_val_loss)
        #wandb.log({"learning_rate": scheduler.get_last_lr[0]})
        #print(scheduler.get_last_lr())

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), "trained_model_best.pth")
            print(f"New best model found and saved with validation loss: {best_val_loss:.8f}")

    epoch_duration = time.time() - epoch_start_time
    elapsed_time = time.time() - start_time
    estimated_total_time = elapsed_time / (epoch + 1) * num_epochs
    estimated_time_left = estimated_total_time - elapsed_time
    
    wandb.log({
        "train_loss": avg_train_loss,
    }, step=epoch)
    if epoch > 20:
        wandb.log({
            "time_left": round(estimated_time_left)
        }, step=epoch)

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_train_loss:.8f} Time Left: {estimated_time_left:.2f} seconds")


Epoch 1/10000, Training Loss: 0.17031344 Time Left: 6499.27 seconds
Epoch 2/10000, Training Loss: 0.17014137 Time Left: 5128.78 seconds
Epoch 3/10000, Training Loss: 0.16996779 Time Left: 4690.53 seconds
Epoch 4/10000, Training Loss: 0.16978994 Time Left: 4457.92 seconds
Epoch 5/10000, Validation Loss: 0.16949005
New best model found and saved with validation loss: 0.16949005
Epoch 5/10000, Training Loss: 0.16960666 Time Left: 4460.14 seconds
Epoch 6/10000, Training Loss: 0.16941737 Time Left: 4347.66 seconds
Epoch 7/10000, Training Loss: 0.16921888 Time Left: 4257.12 seconds
Epoch 8/10000, Training Loss: 0.16900895 Time Left: 4187.25 seconds
Epoch 9/10000, Training Loss: 0.16878474 Time Left: 4134.78 seconds
Epoch 10/10000, Validation Loss: 0.16837704
New best model found and saved with validation loss: 0.16837704
Epoch 10/10000, Training Loss: 0.16853986 Time Left: 4163.89 seconds
Epoch 11/10000, Training Loss: 0.16826937 Time Left: 4119.19 seconds
Epoch 12/10000, Training Loss: 0.16

In [ ]:
# Save the trained model
wandb.finish()
torch.save(model.state_dict(), "trained_model.pth")
print("Training complete. Model saved to trained_model.pth")
print(f"Best validation loss: {best_val_loss}")